In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from os.path import exists
import json
import scipy.stats as stats
import math
from sklearn.model_selection import train_test_split
import seaborn as sb
from sklearn.ensemble import RandomForestClassifier as rf
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
import sklearn.linear_model as lm
from sklearn.tree import DecisionTreeClassifier as tree
from sklearn.neighbors import KNeighborsClassifier as knn
from xgboost import XGBClassifier as xgb
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB as gnb
from sklearn.ensemble import VotingClassifier
import sklearn.model_selection as ms
import sklearn.metrics as sm
import joblib
%autosave 5

In [ ]:
def retrieveModelsBasedOnModelType(modelType):
    if modelType == 'tree':
        gridmodel = tree(random_state=51)
        finalmodel = tree(random_state=51)
    elif modelType == 'forest':
        gridmodel = rf(random_state=51)
        finalmodel = rf(random_state=51)
    elif modelType == 'knn':
        gridmodel = knn()
        finalmodel = knn()
    elif modelType == 'xgboost':
        gridmodel = xgb(random_state=51)
        finalmodel = xgb(random_state=51)
    elif modelType == 'svm':
        gridmodel = SVC(random_state=51)
        finalmodel = SVC(random_state=51)
    else:
        raise Exception("modelType Value not considered. Please choose from ['tree','forest','knn','xgboost','svm']")
    return gridmodel,finalmodel

In [ ]:
def fitModelWithGridSearch(searchParams,XTrain,yTrain,modelType):
    gridmodel,finalmodel = retrieveModelsBasedOnModelType(modelType)
    modelGridSearch = ms.GridSearchCV(gridmodel, param_grid=searchParams,scoring='accuracy',cv=6)
    modelGridSearch.fit(XTrain,yTrain)
    finalmodel.set_params(**modelGridSearch.best_params_)
    return finalmodel
    

In [ ]:
def makeData():
    df = pd.read_csv("../Data/Raw/ACME-HappinessSurvey2020.csv")
    y = df[["Y"]]
    X = df.drop("Y",axis=1)
    XTrain,XTest,yTrain,yTest = train_test_split(X, y, test_size=0.15,random_state=51)
    happinessTrain = XTrain.copy()
    happinessTrain['Y'] = yTrain
    happinessTrain.to_csv('../Data/Processed/happinessTrain.csv',index=False)
    happinessTest = XTest.copy()
    happinessTest['Y'] = yTest
    happinessTest.to_csv('../Data/Processed/happinessTest.csv',index=False)
    yTrain = yTrain.values.ravel()
    yTest = yTest.values.ravel()
    return XTrain,XTest,yTrain,yTest

In [ ]:
def loadData():
    if exists("../Data/Processed/happinessTrain.csv"):
        happinessTrain = pd.read_csv("../Data/Processed/happinessTrain.csv")
        happinessTest = pd.read_csv("../Data/Processed/happinessTest.csv")
        yTrain = happinessTrain[["Y"]].values.ravel()
        XTrain = happinessTrain.drop("Y",axis=1)
        yTest = happinessTest[["Y"]].values.ravel()
        XTest = happinessTest.drop("Y",axis=1)
    else:
        XTrain,XTest,yTrain,yTest = makeData()
    return XTrain,XTest,yTrain,yTest

In [ ]:
def printScore(model,X,y,dataSetType):
    print(f"{dataSetType} accruacy: {model.score(X,y)}")

In [ ]:
def saveModel(model,modelName):
    joblib.dump(value=model, filename=f"../Models/{modelName}.pkl")

In [ ]:
def main():
    np.random.seed(51)
    XTrain,XTest,yTrain,yTest = loadData()
    
    nRows = XTrain.shape[0]
    sqrtNRows = int(math.sqrt(nRows))
    log2NRows = int(math.log2(nRows))
    possibleThirdGeometricTerm1 = int((sqrtNRows ** 2)/log2NRows)
    possibleThirdGeometricTerm2 = int((log2NRows ** 2)/sqrtNRows)
    suggestedMaxKRange = [possibleThirdGeometricTerm1,possibleThirdGeometricTerm2]
    kRange = [int(x) for x in np.linspace(5,max(suggestedMaxKRange),10)]
    
    treeParams = {
        "max_depth":[2,3],
        "max_features":[2,3],
        "criterion": ["gini","entropy"]
    }
    forestParams = {
        "n_estimators": [100,150,200,250,300],
        "max_depth":[2,3],
        "max_features":[2,3],
        "criterion": ["gini","entropy"]
    }
    knnParams = {
        "n_neighbors": kRange
    }
    xgbParams = {
        "learning_rate": list(np.linspace(.1,1,10))
    }
    svmParams = {
        "kernel": ["linear","rbf","poly","sigmoid"],
        "gamma": ["auto","scale"]
    }
    
    logModel = lm.LogisticRegression(max_iter=1e9)
    logPipe = make_pipeline(StandardScaler(), logModel)
    gnbModel = gnb()
    
    estimators = [
        ("logModel",logPipe),
        ("naiveBayes",gnbModel),
        ("tree",fitModelWithGridSearch(treeParams,XTrain,yTrain,'tree')),
        ("forest",fitModelWithGridSearch(forestParams,XTrain,yTrain,'forest')),
        ("knn",fitModelWithGridSearch(knnParams,XTrain,yTrain,'knn')),
        ("xgboost",fitModelWithGridSearch(xgbParams,XTrain,yTrain,'xgboost')),
        ("svm",fitModelWithGridSearch(svmParams,XTrain,yTrain,'svm'))
    ]
    
    goodModels = []
    
    for est in estimators:
        modName = est[0]
        mod = est[1]
        mod.fit(XTrain,yTrain)
        print(modName)
        printScore(mod,XTrain,yTrain,"Training")
        printScore(mod,XTest,yTest,"Testing")
        if mod.score(XTest,yTest) > 0.73:
            saveModel(mod,modName)
            goodModels.append(modName)
    
    goodModelsDictionary = {
        "goodModels": goodModels
    }
    
    with open('../Models/goodModelsDictionary.json', 'w') as fp:
        json.dump(goodModelsDictionary, fp)

In [ ]:
main()